## Apply the Shoreline 

This notebook applies the posterior parameter samples to a few basic questions in the conclusions of Berta-Thompson et al. (2025).

In [ ]:
from shoreline import * 

We'll use the "all planets, any type of atmosphere" shoreline, as it's the most constrained. 

In [ ]:
po = az.from_netcdf('posteriors/all-any-uncertainties=True-numpyro.nc').posterior
po
    

## How much bigger would Mercury need to be to retain an atmosphere?

In [ ]:
m = SolarSystem()['Mercury']
m.semimajoraxis(), m.relative_insolation()

In [ ]:
f_mercury = m.relative_insolation()
f_0 = 10**po['log_f_0']
p = po['p']
po['mercury_escape_velocity'] = (f_mercury/f_0)**(1/p)
v_ratio = po['mercury_escape_velocity']/m.relative_escape_velocity()
po['mercury_r_ratio'] = v_ratio**(2/(3.375 - 1))
po['mercury_r'] =po['mercury_r_ratio']*m.radius()[0].value

## How much hotter could Venus be before losing its atmosphere? 

In [ ]:
v = SolarSystem()['Venus']
v.relative_escape_velocity()

In [ ]:

po['venus-flux'] = f_0*v.relative_escape_velocity()**p
po['venus-logflux'] = np.log10(po['venus-flux'])
po['venus-semimajor'] = po['venus-flux']**(-0.5)


## How faint a star could Earth orbit and stay habitable? 

In [ ]:
q = po['q'] 
#po['earth-luminosity-limit'] = (1/f_0)**(1/q)
#po['earth-log-luminosity-limit'] = np.log10(po['earth-luminosity-limit'])
po['earth-log-luminosity-limit'] = -po['log_f_0']/po['q']
po['earth-luminosity-limit'] = 10**po['earth-log-luminosity-limit']
R = 1.5
v = R**(1/0.842)
po['earth-and-a-half-luminosity-limit'] = (1/f_0*1/v**p)**(1/q)
po['earth-and-a-half-log-luminosity-limit'] = np.log10(po['earth-and-a-half-luminosity-limit'])


## How sudden is the transition? 

In [ ]:
po['w95-f'] = 5.88*po['w']
po['w95-v'] = po['w95-f']/po['p']
po['w95-L'] = po['w95-f']/po['q']


## How likely are the Rocky Worlds targets to have atmospheres? 

In [ ]:
A = load_organized_populations(label='everything', subpop='transit', subset='all', kind='any')
t = A['all-any']['everything']['transit']
t


In [ ]:
sampled_parameters = {k:jnp.array(po[k]) for k in ['log_f_0', 'p', 'q', 'ln_w']}
for k in ['LTT1445Ac', 'GJ 3929 b', 'LTT 1445 A b', 'LHS 1140 b']:
    e = t[k]
    p_samples = probability_of_atmosphere(log_f=jnp.log10(e.relative_insolation()[0]), 
                              log_L=jnp.log10(e.stellar_luminosity().to_value('L_sun')[0]),
                              log_v=jnp.log10(e.relative_escape_velocity()[0]),
                              **sampled_parameters
                              )
    po[f'probability-{k}'] = ['chain','draw'], p_samples*100


In [ ]:
az.summary(po, stat_focus='median')

## (write these all out in LaTeX variables we can use)

In [ ]:
with open('posteriors-applications.tex', 'w') as f:
    lines = latexify(po, label='apply')
    f.writelines(lines)

In [ ]:
!cat posteriors-applications.tex